## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [3]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [4]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [5]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM
0,100002,1,0,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,1,0,1,0,False
1,100003,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,0,0,0,0,0,1,0,False
2,100004,0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,False
3,100006,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,0,0,0,0,0,False
4,100007,0,0,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,False


In [66]:
app_train["equal_width"] = pd.cut(app_train["AMT_INCOME_TOTAL"], 30)


In [67]:
app_train["equal_freq"] = pd.qcut(app_train["OWN_CAR_AGE"], 10)

In [68]:
app_train["equal_width"].value_counts()

(-91324.35, 3924795.0]        307501
(3924795.0, 7823940.0]             6
(15622230.0, 19521375.0]           1
(7823940.0, 11723085.0]            1
(11723085.0, 15622230.0]           1
(113100855.0, 117000000.0]         1
(89705985.0, 93605130.0]           0
(85806840.0, 89705985.0]           0
(105302565.0, 109201710.0]         0
(101403420.0, 105302565.0]         0
(97504275.0, 101403420.0]          0
(93605130.0, 97504275.0]           0
(19521375.0, 23420520.0]           0
(23420520.0, 27319665.0]           0
(27319665.0, 31218810.0]           0
(31218810.0, 35117955.0]           0
(35117955.0, 39017100.0]           0
(39017100.0, 42916245.0]           0
(42916245.0, 46815390.0]           0
(46815390.0, 50714535.0]           0
(50714535.0, 54613680.0]           0
(109201710.0, 113100855.0]         0
(58512825.0, 62411970.0]           0
(62411970.0, 66311115.0]           0
(66311115.0, 70210260.0]           0
(70210260.0, 74109405.0]           0
(74109405.0, 78008550.0]           0
(

In [65]:
app_train["equal_freq"].value_counts()

(-0.001, 2.0]    13266
(9.0, 12.0]      13224
(2.0, 4.0]       11927
(7.0, 9.0]       10907
(22.0, 91.0]     10342
(4.0, 6.0]        9977
(14.0, 17.0]      9834
(12.0, 14.0]      9160
(17.0, 22.0]      8521
(6.0, 7.0]        7424
Name: equal_freq, dtype: int64

絕大多數的人的 AMT_INCOME_TOTAL 都在(-91324.35, 3924795.0] 這個級距中 <b
在 OWN_CAR_AGE 這段比較奇特的地方是 (22.0, 91.0] 這段區間有車的人, 居然跟其他未成年的級距有差不多的人數
意思是大家小時候都有車的意思嗎..